# Dirichlet Distribution

*A multinomial generalization of Beta distribution*


$f(x_1,\dots,x_k;\alpha_1,\dots,\alpha_K) = \dfrac{1}{B(\alpha)}\displaystyle \prod_{i=1}^{K} x_i^{\alpha_i-1}$<br><br>

$B(\alpha) = \dfrac{\displaystyle \prod_{i=1}^{K}\Gamma(\alpha_i)}{\Gamma(\displaystyle \sum_{i=1}^{K}\alpha_i)}$

# Dirichlet Process

Dirichlet processes (after Peter Gustav Lejeune Dirichlet) are a family of stochastic processes whose realizations are probability distributions

Dirichlet Process is a distribution over distribution parameterized by $\alpha$ (dispersion parameter) and $G$(Base Distribution)

Dirichlet Process can be demostrated by three models:<br>
1.*Chinese Restaurant Process*<br>
2.*Polya Urn Process*<br>
3.*Stick Breaking Process*<br>

Algorithm for simulation of generation of $X_1,X_2,\dots,$ by Dirichlet Process:

1.Draw $X_1$ from the base distribution $G$<br>
2.For $n>1$:<br>

-With the probability $\dfrac{\alpha}{n+\alpha}$ sample $X_n$ from $G$

-With the probability $\dfrac{n_x}{n+\alpha}$ set $X_x=x,$  n_x := |${$:X_j=x$ and $j<n$| where $|.|$ denotes the number of elements in the set

# Properties of DP

Let DP($\alpha$, $G$ ) be the Dirichlet Process with dispersion parameter $\alpha$ and base distribution $G$, then<br><br>

<center>$\mathbf{E}_{DP(\alpha,G)}[x] = \mathbf{E}_G[x]$<br><br>
As $\alpha \rightarrow \infty,  DP(\alpha,G) \rightarrow G$</center>


# Gibbs Sampling for DPGMM

*Base distribution is Gaussain distribution*
<br>
<center>$p(z_i = k | z_{-i}) \equiv p(z_i | z_1 \dots z_{i-1},z_{i+1},\dots z_m)$</center>

<center>$p(z_i = k|z_{-i},\vec{x},{\theta_k},\alpha) $<br><br>
$= p(z_i = k|z_{-i},x_i,\vec{x},\theta_k,\alpha) $<br><br>
$= p(z_i = k|z_{-i},\alpha)p(x_i|\theta_k,\vec{x})$</center>

\begin{cases}
({\dfrac{n_k}{n+\alpha}})\mathcal{N}(x,\dfrac{n_x}{n+1},\mathbf{1}),\space existing\\
({\dfrac{\alpha}{n+\alpha}})\mathcal{N}(x,0,1)),\space new
\end{cases}

where,<br>
$z_i$ is the group assignment of $x_i$<br>
$k$ is the cluster label<br>
$\alpha$ is the dispersion parameter

In [1]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.tri as tri
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact
from scipy.stats import dirichlet, multivariate_normal
from scipy import stats
import math
import pandas as pd
from sklearn import datasets

In [2]:
def Dir_Dist(a1, a2, a3):
    alpha = np.array([a1,a2,a3])
    theta = stats.dirichlet(alpha).rvs(1000)
    fig = plt.figure(figsize=(8, 8), dpi=100)
    ax = plt.gca(projection='3d')
    plt.title(r'$\alpha$ = {}'.format(alpha))
    ax.scatter(theta[:, 0], theta[:, 1], theta[:, 2])
    ax.view_init(azim=30)
    ax.set_xlabel(r'$\theta_1$')
    ax.set_ylabel(r'$\theta_2$')
    ax.set_zlabel(r'$\theta_3$')
    plt.show()

In [3]:
interact(Dir_Dist, a1=(0.1,2),a2=(0.1,2),a3=(0.1,2))

interactive(children=(FloatSlider(value=1.05, description='a1', max=2.0, min=0.1), FloatSlider(value=1.05, des…

<function __main__.Dir_Dist(a1, a2, a3)>

In [4]:
def dirichlet_process(h_0, alpha):
    """
    Truncated dirichlet process.
    :param h_0: (scipy distribution)
    :param alpha: (flt)
    :param n: (int) Truncate value.
    """
    n = max(int(5 * alpha + 2), 500)  # truncate the values. 
    pi = stats.beta(1, alpha).rvs(size=n)
    pi[1:] = pi[1:] * (1 - pi[:-1]).cumprod()  # stick breaking process
    theta = h_0(size=n)  # samples from original distribution
    return pi, theta
        
def plot_normal_dp_approximation(alpha, n=3):
    pi, theta = dirichlet_process(stats.norm.rvs, alpha)
    x = np.linspace(-4, 4, 100)
    
    plt.figure(figsize=(14, 4))
    plt.suptitle(r'Three samples from DP($\alpha$). $\alpha$ = {}'.format(alpha))
    plt.ylabel(r'$\pi$')
    plt.xlabel(r'$\theta$')
    pltcount = int('1' + str(n) + '0')
    
    for i in range(n):
        pltcount += 1
        plt.subplot(pltcount)
        pi, theta = dirichlet_process(stats.norm.rvs, alpha)
        pi = pi * (stats.norm.pdf(0) / pi.max())
        plt.vlines(theta, 0, pi, alpha=0.5)
        plt.ylim(0, 1)
        plt.plot(x, stats.norm.pdf(x))
        print(len(theta),len(pi))
    plt.show()

In [5]:
interact(plot_normal_dp_approximation, alpha=(1, 50), n=(2,3))

interactive(children=(IntSlider(value=25, description='alpha', max=50, min=1), IntSlider(value=3, description=…

<function __main__.plot_normal_dp_approximation(alpha, n=3)>